In [9]:
import numpy as np
import cv2
import pandas as pd
from PIL import Image

In [2]:
maplist = pd.read_csv('datasets/maps/maplist.csv', index_col=0)
maplist.columns=['modo','mapa']

maplist

,modo,mapa
0,knockout,X Marks the Spot
1,knockout,Riverside
2,knockout,Out in the Open
3,knockout,New Perspective
4,knockout,Goldarm Gulch
5,knockout,Flaring Phoenix
6,knockout,Belle's Rock
7,hotZone,Split
8,hotZone,Ring of Fire
9,hotZone,Quarter Pounder


In [59]:
# contar arbustos por mapa

bushList = []

for i, mapname in enumerate(maplist['mapa']):
	# Carga la imagen donde se buscará
	img_rgb = cv2.imread("resources/maps/" + mapname + ".png", 0)

	# Carga la imagen a buscar
	img_gray = cv2.imread("resources/maps/bush_detection/grassred3.png", 0)

	# Utiliza el método matchTemplate de OpenCV para buscar la imagen en la otra
	res = cv2.matchTemplate(img_rgb, img_gray, cv2.TM_CCOEFF_NORMED)

	# Utiliza la función threshold para obtener una máscara con los puntos donde se encontró una coincidencia
	loc = np.where(res >= 0.9)

	puntos=list(zip(*loc[::-1]))

	def filtrar_puntos_por_distancia(puntos, min_dist):
		puntos_filtrados = []
		for point in puntos:
			is_valid = True
			point = np.array(point)
			for point_filtrado in puntos_filtrados:
				point_filtrado = np.array(point_filtrado)
				dist = np.linalg.norm(point - point_filtrado)
				if dist < min_dist:
					is_valid = False
					break
			if is_valid:
				puntos_filtrados.append(point)
		return puntos_filtrados

	puntos = filtrar_puntos_por_distancia(puntos, 30)

	for point in puntos:
		cv2.drawMarker(img_rgb, point, (0,0,255), cv2.MARKER_DIAMOND, 20, 2)

	# Muestra el resultado 
	# print("La imagen aparece " + str(len(puntos)) + " veces en la imagen")
	# cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)
	
	puntos_centro = [p for p in puntos if (p[0]>150 and p[0]<540 and p[1]<720 and p[1]>330)]
	puntos_centro

	for point in puntos_centro:
		cv2.drawMarker(img_rgb, point, (255,0,0), cv2.MARKER_CROSS, 20, 2)

	cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)

	bushList.append({'event.map': mapname, 'bushes': len(puntos), 'bushes_center': len(puntos_centro)})

bushList = pd.DataFrame(bushList)
bushList.to_csv('datasets/maps/bushList.csv')
bushList

,mapname,bushes,bushes_center
0,X Marks the Spot,78,46
1,Riverside,168,14
2,Out in the Open,71,30
3,New Perspective,102,2
4,Goldarm Gulch,123,12
5,Flaring Phoenix,113,36
6,Belle's Rock,36,0
7,Split,94,14
8,Ring of Fire,114,17
9,Quarter Pounder,31,4


In [5]:
# contar arbustos por mapa

barrierList = []

barrierTemplates = ['barriertop1.png','barriertop2.png','barriertop3.png','barriertop4.png','barriertop5.png','barriertop6.png','barriertop7.png'
	,'barriertop8.png','barriertop9.png','barriertop10.png','barriertop11.png','barriertop12.png','barriertop13.png','barriertop14.png','barriertop15.png']

#FALTA COMPLEMENTAR BARRIERTOPS FALTANTES

for i, mapname in enumerate(maplist['mapa']):
	puntos = []
	# Carga la imagen donde se buscará
	img_rgb = cv2.imread("resources/maps/" + mapname + ".png", 0)

	for i, template in enumerate(barrierTemplates):
		# Carga la imagen a buscar
		img_gray = cv2.imread("resources/maps/barrier_detection/" + template, 0)

		# Utiliza el método matchTemplate de OpenCV para buscar la imagen en la otra
		res = cv2.matchTemplate(img_rgb, img_gray, cv2.TM_CCOEFF_NORMED)

		# Utiliza la función threshold para obtener una máscara con los puntos donde se encontró una coincidencia
		loc = np.where(res >= 0.9)

		puntos = puntos + list(zip(*loc[::-1]))

	def filtrar_puntos_por_distancia(puntos, min_dist):
		puntos_filtrados = []
		for point in puntos:
			is_valid = True
			point = np.array(point)
			for point_filtrado in puntos_filtrados:
				point_filtrado = np.array(point_filtrado)
				dist = np.linalg.norm(point - point_filtrado)
				if dist < min_dist:
					is_valid = False
					break
			if is_valid:
				puntos_filtrados.append(point)
		return puntos_filtrados

	puntos = filtrar_puntos_por_distancia(puntos, 30)

	for point in puntos:
		cv2.drawMarker(img_rgb, point, (0,0,255), cv2.MARKER_DIAMOND, 20, 2)

	# Muestra el resultado
	# print("La imagen aparece " + str(len(puntos)) + " veces en la imagen")
	# cv2.imwrite("resources/maps/bush_detection/" + mapname + ".png", img_rgb)
	
	puntos_centro = [p for p in puntos if (p[0]>150 and p[0]<540 and p[1]<720 and p[1]>330)]
	puntos_centro

	for point in puntos_centro:
		cv2.drawMarker(img_rgb, point, (255,0,0), cv2.MARKER_CROSS, 20, 2)

	cv2.imwrite("resources/maps/barrier_detection/" + mapname + ".png", img_rgb)

	barrierList.append({'event.map': mapname, 'barriers': len(puntos), 'barriers_center': len(puntos_centro)})

barrierList = pd.DataFrame(barrierList)
barrierList.to_csv('datasets/maps/barrierList.csv')
barrierList

,mapname,barriers,barriers_center
0,X Marks the Spot,39,15
1,Riverside,90,8
2,Out in the Open,51,12
3,New Perspective,56,18
4,Goldarm Gulch,74,32
5,Flaring Phoenix,97,22
6,Belle's Rock,102,24
7,Split,59,33
8,Ring of Fire,16,0
9,Quarter Pounder,65,16


In [110]:
# water pixelcount
diccionario_colores = [
	{'mapname': "Belle's Rock", 'color': (0, 36, 80, 255)},
	{'mapname': 'Bridge Too Far', 'color': (31, 151, 223, 255)},
	{'mapname': 'Canal Grande', 'color': (0, 147, 93, 255)},
	{'mapname': 'Controller Chaos', 'color': (31, 151, 223, 255)},
	{'mapname': 'Dueling Beetles', 'color': (0, 147, 93, 255)},
	{'mapname': 'Flaring Phoenix', 'color': (31, 151, 223, 255)},
	{'mapname': 'Goldarm Gulch', 'color': (31, 151, 223, 255)},
	{'mapname': 'Kaboom Canyon', 'color': (31, 151, 223, 255)},
	{'mapname': 'New Perspective', 'color': (0, 147, 93, 255)},
	{'mapname': 'Open Zone', 'color': (0, 147, 93, 255)},
	{'mapname': 'Out in the Open', 'color': (0, 147, 93, 255)},
	{'mapname': 'Parallel Plays', 'color': (31, 151, 223, 255)},
	{'mapname': 'Quarter Pounder', 'color': (0, 147, 93, 255)},
	{'mapname': 'Ring of Fire', 'color': (0, 36, 80, 255)},
	{'mapname': 'Riverside', 'color': (0, 147, 93, 255)},
	{'mapname': 'Safe Zone', 'color': (0, 147, 93, 255)},
	{'mapname': 'Shooting Star', 'color': (0, 36, 80, 255)},
	{'mapname': 'Snaked Assault', 'color': (0, 147, 93, 255)},
	{'mapname': 'Tornado Ring', 'color': (31, 151, 223, 255)},
	{'mapname': 'X Marks the Spot', 'color': (31, 151, 223, 255)},
	]

waterList = []

for i, dict in enumerate(diccionario_colores):

	im = Image.open("resources/maps/" + dict['mapname'] + ".png")

	pixels = np.array(im)

	coordenadas = np.where(np.all(pixels == dict['color'], axis=-1))

	count = len(coordenadas[0])

	total_pixels = im.size[0] * im.size[1]

	waterList.append({'event.map': dict['mapname'], 'waterProp': count / total_pixels})

waterList = pd.DataFrame(waterList)
waterList.to_csv('datasets/maps/waterList.csv')
waterList

,mapname,waterProp
0,Belle's Rock,0.014667
1,Bridge Too Far,0.110888
2,Canal Grande,0.111227
3,Controller Chaos,0.017400
4,Dueling Beetles,0.070509
5,Flaring Phoenix,0.013941
6,Goldarm Gulch,0.038716
7,Kaboom Canyon,0.022882
8,New Perspective,0.025382
9,Open Zone,0.025001
